# Generating 1000+ Models

In [1]:
import sys
import socket
computer_name = socket.gethostname()
if computer_name == 'Alexs-MacBook-Pro.local':
    base_path = "/Users/alexsutherland/Documents/Programming/Python/Kaggle/Titanic---2015"
else:    
    base_path = 'C:\Users\Lundi\Documents\Programming\Python\Kaggle\Titanic - 2015'
sys.path.append(base_path)
sys.path.append(base_path + "\Stacked Generalization")

import TitanicPreprocessor as tp
import TitanicPredictor as tpred
import metaLearning as metaLearn
meta_learn = metaLearn.metaLearning()
import sklearn.ensemble as skl_ensemble
import sklearn.linear_model as skl_lm
import sklearn.grid_search as skl_gs
import sklearn.cross_validation as skl_cv
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

/Users/alexsutherland/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## Random Forest

In [2]:
rf_clf = skl_ensemble.RandomForestClassifier()

param_grid = [
    {'n_estimators': [10,100, 1000], 'criterion':['gini','entropy'], 'max_depth':[None,1,2,4,6,8,10], 'max_features':[1,2,4] }
    ]

rf_param_grid = skl_gs.ParameterGrid(param_grid = param_grid)

In [3]:
meta_learn.generateModelPredictions(rf_clf, rf_param_grid)

In [5]:
len(meta_learn.model_list)

226

## Logistic Regression

In [4]:
lr_clf = skl_lm.LogisticRegression()

param_grid = [
    {'penalty':['l1','l2'], 'C':np.logspace(-10,10,num=50), 'solver': ['liblinear'] }
    ]

lr_param_grid = skl_gs.ParameterGrid(param_grid = param_grid)

meta_learn.generateModelPredictions(lr_clf, lr_param_grid)

## AdaBoost with DT stumps

In [7]:
import sklearn.tree as skl_tree

In [10]:
dt_stump_clf = skl_tree.DecisionTreeClassifier(max_depth=1)
ada_dt_stump_clf = skl_ensemble.AdaBoostClassifier(base_estimator=dt_stump_clf)

param_grid = [
    {'n_estimators':[50, 100, 500, 1000], 'learning_rate':[0.001, 0.1, 1.0] }
    ]

ada_dt_stump_param_grid = skl_gs.ParameterGrid(param_grid = param_grid)

meta_learn.generateModelPredictions(ada_dt_stump_clf, ada_dt_stump_param_grid)

In [11]:
len(meta_learn.model_list)

235

## Random Subspace Method with kNN

In [2]:
import sklearn.neighbors as skl_neighbors

In [8]:
for k in np.arange(2,30):
    for repeat in xrange(2):
        knn_subspace_clf = skl_ensemble.BaggingClassifier(skl_neighbors.KNeighborsClassifier(n_neighbors=k), 
                                   bootstrap=False, 
                                   bootstrap_features=True, 
                                   max_features=0.7, 
                                    random_state=repeat)
        meta_learn.generateSingleModelPredictions(knn_subspace_clf)

In [12]:
meta_learn.model_list[len(meta_learn.model_list)-1]

"BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',\n           metric_params=None, n_jobs=1, n_neighbors=29, p=2,\n           weights='uniform'),\n         bootstrap=False, bootstrap_features=True, max_features=0.7,\n         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,\n         random_state=1, verbose=0, warm_start=False)"

In [4]:
knn_subspace_clf = skl_neighbors.KNeighborsClassifier(n_neighbors=3)

In [8]:
knn_subspace_clf_string = str(knn_subspace_clf)
knn_subspace_clf_string

"KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',\n           metric_params=None, n_jobs=1, n_neighbors=3, p=2,\n           weights='uniform')"

In [15]:
exec 'import sklearn; x = sklearn.neighbors.'+knn_subspace_clf_string

In [23]:
meta_learn.data['X_probe']

,Pclass,Age,SibSp,Parch,Fare,Is_male,Embarked Q,Embarked S
495,3,26.197559,0,0,14.4583,1,0,0
648,3,28.326038,0,0,7.5500,1,0,1
278,3,7.000000,4,1,29.1250,1,1,0
31,1,31.671301,1,0,146.5208,0,0,0
255,3,29.000000,0,2,15.2458,0,0,0
298,1,42.477347,0,0,30.5000,1,0,1
609,1,40.000000,0,0,153.4625,0,0,1
318,1,31.000000,0,2,164.8667,0,0,1
484,1,25.000000,1,0,91.0792,1,0,0
367,3,23.100909,0,0,7.2292,0,0,0
